In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [4]:
books


isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5192  9788172235222  8172235224   
5193  9788173031014  8173031010   
5194  9788179921623  817992162X   
5195  9788185300535  8185300534   
5196  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5192                                  Mistaken Identity   
5193                                Journey to the East   
5194  The Monk Who Sold His Ferrari: A Fable About F...   
5195                                          I Am that   
5196                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5192                              Nayantara Sahgal   
5193                                 Hermann Hesse   
5194                                  Robin Sharma   
5195  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5196                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5192        Indic fiction (English)   
5193              Adventure stories   
5194               Health & Fitness   
5195                     Philosophy   
5196                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5192  http://books.google.com/books/content?id=q-tKP...   
5193  http://books.google.com/books/content?id=rq6JP...   
5194  http://books.google.com/books/content?id=c_7mf...   
5195  http://books.google.com/books/content?id=Fv_JP...   
5196  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5192  On A Train Journey Home To North India After L...          2003.0   
5193  This book tells the tale of a man who goes on ...          2002.0   
5194  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5195  This collection of the timeless teachings of o...          1999.0   
5196  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
1               3.83      241.0         5164.0   
2    

In [5]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [6]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [7]:
documents[5]

Document(metadata={'source': 'tagged_description.txt'}, page_content="9780006380832 Until Vasco da Gama discovered the sea-route to the East in 1497-9 almost nothing was known in the West of the exotic cultures and wealth of the Indian Ocean and its peoples. It is this civilization and its destruction at the hands of the West that Richard Hall recreates in this book. Hall's history of the exploration and exploitation by Chinese and Arab travellers, and by the Portuguese, Dutch and British alike is one of brutality, betrayal and colonial ambition.")

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()  # ✅ load variables from .env file

api_key = os.getenv("GOOGLE_API_KEY")
print(f"API Key loaded: {api_key[:5]}...")  # should now work

API Key loaded: AIzaS...


In [9]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db_books = Chroma.from_documents(
    documents,
    embedding=embedding)

In [10]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='8f3f1b36-ea67-4e37-a5b8-a552173acec9', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='0afc0737-6bdd-4fd8-b1a8-18ea759bdbf3', metadata={'source': 'tagged_description.txt'}, page_content="9781406904833 This is a reproduction of the original artefact. Generally these books are created from careful scans of the original. This allows us to preserve the book accurately and present it in the way the author intended. Since the original versions are generally quite old, there may occasionally be certain imperfections within these reproductions. We're happy to make these classics available again for future generations to enjoy!"),
 Document(id='bc1b3aff-6743-4832-b66d-ceb0e4e46b

In [11]:
books

isbn13      isbn10  \
0     9780002005883  0002005883   
1     9780002261982  0002261987   
2     9780006178736  0006178731   
3     9780006280897  0006280897   
4     9780006280934  0006280935   
...             ...         ...   
5192  9788172235222  8172235224   
5193  9788173031014  8173031010   
5194  9788179921623  817992162X   
5195  9788185300535  8185300534   
5196  9789027712059  9027712050   

                                                  title  \
0                                                Gilead   
1                                          Spider's Web   
2                                        Rage of angels   
3                                        The Four Loves   
4                                   The Problem of Pain   
...                                                 ...   
5192                                  Mistaken Identity   
5193                                Journey to the East   
5194  The Monk Who Sold His Ferrari: A Fable About F...   
5195                                          I Am that   
5196                           The Berlin Phenomenology   

                                           authors  \
0                               Marilynne Robinson   
1                  Charles Osborne;Agatha Christie   
2                                   Sidney Sheldon   
3                              Clive Staples Lewis   
4                              Clive Staples Lewis   
...                                            ...   
5192                              Nayantara Sahgal   
5193                                 Hermann Hesse   
5194                                  Robin Sharma   
5195  Sri Nisargadatta Maharaj;Sudhakar S. Dikshit   
5196                 Georg Wilhelm Friedrich Hegel   

                         categories  \
0                           Fiction   
1     Detective and mystery stories   
2                           Fiction   
3                    Christian life   
4                    Christian life   
...                             ...   
5192        Indic fiction (English)   
5193              Adventure stories   
5194               Health & Fitness   
5195                     Philosophy   
5196                        History   

                                              thumbnail  \
0     http://books.google.com/books/content?id=KQZCP...   
1     http://books.google.com/books/content?id=gA5GP...   
2     http://books.google.com/books/content?id=FKo2T...   
3     http://books.google.com/books/content?id=XhQ5X...   
4     http://books.google.com/books/content?id=Kk-uV...   
...                                                 ...   
5192  http://books.google.com/books/content?id=q-tKP...   
5193  http://books.google.com/books/content?id=rq6JP...   
5194  http://books.google.com/books/content?id=c_7mf...   
5195  http://books.google.com/books/content?id=Fv_JP...   
5196  http://books.google.com/books/content?id=Vy7Sk...   

                                            description  published_year  \
0     A NOVEL THAT READERS and critics have been eag...          2004.0   
1     A new 'Christie for Christmas' -- a full-lengt...          2000.0   
2     A memorable, mesmerizing heroine Jennifer -- b...          1993.0   
3     Lewis' work on the nature of love divides love...          2002.0   
4     "In The Problem of Pain, C.S. Lewis, one of th...          2002.0   
...                                                 ...             ...   
5192  On A Train Journey Home To North India After L...          2003.0   
5193  This book tells the tale of a man who goes on ...          2002.0   
5194  Wisdom to Create a Life of Passion, Purpose, a...          2003.0   
5195  This collection of the timeless teachings of o...          1999.0   
5196  Since the three volume edition ofHegel's Philo...          1981.0   

      average_rating  num_pages  ratings_count  \
0               3.85      247.0          361.0   
1               3.83      241.0         5164.0   
2    

In [12]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]


isbn13      isbn10                                title  \
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  \
3747            3.89       16.0          180.0   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged_description  
3747  9780786808069 Children will discover the excit...

In [13]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [14]:
retrieve_semantic_recommendations("A book to teach children about nature")


isbn13      isbn10                                 title  \
334   9780060976118  006097611X              Operation Wandering Soul   
544   9780130235015  0130235016                     Things Fall Apart   
711   9780140621624  0140621628                  The Railway Children   
855   9780143037392  0143037390               The Read-aloud Handbook   
990   9780192854520  0192854526  Augustine: A Very Short Introduction   
991   9780192862099  019286209X                   The Origins of Life   
1169  9780310929550  0310929555             Zondervan NIV Study Bible   
1222  9780312319731  0312319738                              Dominion   
1288  9780312890216  0312890214                       The Starry Rift   
1545  9780345477606  034547760X                             Star Wars   

                                  authors                   categories  \
334                        Richard Powers                      Fiction   
544           Chinua Achebe;Sandra Widner        Igbo (African people)   
711                             E. Nesbit                      Fiction   
855                          Jim Trelease  Language Arts & Disciplines   
990                        Henry Chadwick    Biography & Autobiography   
991     John Maynard Smith;Eörs Szathmáry                      Science   
1169  Kenneth L. Barker;Donald W. Burdick                       Bibles   
1222                       Matthew Scully                       Nature   
1288                        James Tiptree                      Fiction   
1545       Ann Margaret Lewis;Helen Keier                      Fiction   

                                              thumbnail  \
334   http://books.google.com/books/content?id=nIGIm...   
544                                                 NaN   
711   http://books.google.com/books/content?id=fFesd...   
855   http://books.google.com/books/content?id=B2_yU...   
990   http://books.google.com/books/content?id=QnrRB...   
991   http://books.google.com/books/content?id=nHDbB...   
1169  http://books.google.com/books/content?id=WvYLG...   
1222  http://books.google.com/books/content?id=_htG-...   
1288                                                NaN   
1545  http://books.google.com/books/content?id=InPJD...   

                                            description  published_year  \
334   Highly imaginative and emotionally powerful, t...          1994.0   
544   [This book is] a simple story of a "strong man...          2000.0   
711   When their father is sent away to prison, thre...          1995.0   
855   Explains the importance of reading aloud to ch...          2006.0   
990   Very Short Introductions offer stimulating, ac...          2001.0   
991   'I can recommend this book as a thoroughly int...          2000.0   
1169  #1 Best-selling study Bible in the best-sellin...          2002.0   
1222  Argues for responsible action in the treatment...          2003.0   
1288  This novel set in the far-future and filled wi...          1994.0   
1545  A comprehensive overview of alien beings from ...          2006.0   

      average_rating  num_pages  ratings_count  \
334             3.62      352.0          366.0   
544             3.64      113.0           40.0   
711             4.00      212.0          178.0   
855             4.40      432.0         4122.0   
990             3.74      144.0          219.0   
991             4.11      192.0           41.0   
1169            4.70     2198.0         4092.0   
1222            4.16      448.0         1131.0   
1288            3.82      250.0          220.0   
1545            4.16      227.0          247.0   

                                     title_and_subtitle  \
334                            Operation Wandering Soul   
544               Things Fall Apart: An Adapted Classic   
711                                The Railway Children   
855                             The Read-aloud Handbook   
990                Augustine: A Very Short Introduction   
991   The Ori